In [15]:
from FME.interpolators.finite_difference_interpolator import FiniteDifferenceInterpolator as FDI
from FME.supports.structured_grid import StructuredGrid
from FME.modelling.features.geological_feature import GeologicalFeatureInterpolator
# from FME.modelling.features.faulted_geological_feature import FaultedGeologicalFeature
from FME.visualisation.model_visualisation import LavaVuModelViewer
# from FME.modelling.structural_frame import StructuralFrameBuilder, StructuralFrame
# from FME.modelling.fault.fault_segment import FaultSegment
import numpy as np
import lavavu
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [16]:
dips = pd.read_csv('../Dips.csv',delimiter=';')
points_file_names = ['../APoints.csv','../BPoints.csv','../CPoints.csv','../DPoints.csv']
section_file_names = ['../ASection.csv','../BSection.csv','../CSection.csv','../DSection.csv']

# import all of the csv into the same dataframe
dfs = []
for f in points_file_names:
    dfs.append(pd.read_csv(f,delimiter=';'))
points = pd.concat(dfs,axis=0,ignore_index=True)
# sections
dfs = []
for f in section_file_names:
    dfs.append(pd.read_csv(f,delimiter=';'))
sections = pd.concat(dfs,axis=0,ignore_index=True)
# apoints = pd.read_csv('APoints.csv',delimiter=';')
# asection = pd.read_csv('ASection.csv',delimiter=';')



In [17]:
origin = (548800, 7816600, -11010,)
nsteps = np.array((38,55,30))
# nsteps = np.array((19,28,15))

step_vector = (100,100,90)
grid = StructuredGrid(origin=origin, nsteps=nsteps,step_vector=step_vector)



In [18]:
interpolator = FDI(grid)
stratigraphy_builder = GeologicalFeatureInterpolator(
    interpolator=interpolator,
    name='stratigraphy')
solver = 'lu'
for i, r in points.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in sections.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in dips.iterrows():
    stratigraphy_builder.add_planar_constraint([r['X'],r['Y'],r['Z']],[r['OrientX'],r['OrientY'],r['OrientZ']])
# stratigraphy_builder.add_point([6.1,0.1,1.1],0.)
# stratigraphy_builder.add_point([6.1,0.1,2.1],1.)
# stratigraphy_builder.add_strike_and_dip([1,1,1],90.,0.)
unique = np.unique(stratigraphy_builder.interpolator.get_control_points()[:,3])

In [19]:
stratigraphy = stratigraphy_builder.build(    
    solver='cg',
    dxy=.1,
    dyz=.1,
    dxz= .1,
    dxx= .5,
    dyy= .5,
    dzz= 2,
)


341642 (341642, 62700)
Conjugate gradient converged


In [ ]:
viewer = LavaVuModelViewer(background="white")
viewer.plot_isosurface(stratigraphy, slices=unique, colour='blue')
viewer.plot_isosurface(stratigraphy, slices=[30,125,300], colour='red')

viewer.plot_value_data(stratigraphy.support.interpolator.get_control_points()[:,:3],
                       stratigraphy.support.interpolator.get_control_points()[:,3],
                       'val')
viewer.plot_vector_data(
    stratigraphy.support.interpolator.get_gradient_control()[:,:3],
    stratigraphy.support.interpolator.get_gradient_control()[:,3:],
    'grad')
viewer.interactive()

In [7]:
np.savetxt("fdi_claudius_lg.csv",stratigraphy.support.get_node_values())

In [7]:
x = np.linspace(548800,552500,38)
y = np.linspace(7816600,7822000,55)
z = np.linspace(-11010,-8400,30)

xx,yy,zz = np.meshgrid(x,y,z)

xx=xx.flatten()
yy=yy.flatten()
zz=zz.flatten()
points = np.array([xx,yy,zz]).T
v = stratigraphy.evaluate_value(points)
np.savetxt("fdi_claudius38_55_30.csv",v)

IndexError: index 62700 is out of bounds for axis 0 with size 62700